In [12]:
import numpy as np

# Training input and target
X = np.array([[0.4, -2],
              [0.2, -1],
              [0, 0],
              [-0.2, 1],
              [-0.4, 2],
              [0.3, -2]])

y = np.array([[-10],
              [-5],
              [0],
              [5],
              [10],
              [-9]])

# Sigmoid activation function and derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Network architecture
input_size = 2
hidden_size = 4
output_size = 1
lr = 0.1  # learning rate
epochs = 10000

# Initialize weights and biases
np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size)
b2 = np.zeros((1, output_size))

# Training loop
for epoch in range(epochs):
    # Forward pass
    z1 = np.dot(X, W1) + b1
    a1 = sigmoid(z1)

    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)

    # Compute error
    error = y - a2
    loss = np.mean(error ** 2)

    # Backpropagation
    d_output = error * sigmoid_derivative(a2)
    d_hidden = np.dot(d_output, W2.T) * sigmoid_derivative(a1)

    # Update weights and biases
    W2 += np.dot(a1.T, d_output) * lr
    b2 += np.sum(d_output, axis=0, keepdims=True) * lr
    W1 += np.dot(X.T, d_hidden) * lr
    b1 += np.sum(d_hidden, axis=0, keepdims=True) * lr

    # Print loss occasionally
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


# Test daten
x_test = np.array([[0.4, -2]])

# Test
#output = sigmoid(np.dot(sigmoid(np.dot(x_test, W1) + b1), W2) + b2)
output = (np.dot((np.dot(x_test, W1) + b1), W2) + b2)

print(output)
#print("\nPredictions:\n", output.round())


Epoch 0, Loss: 57.3515
Epoch 1000, Loss: 50.5040
Epoch 2000, Loss: 50.5019
Epoch 3000, Loss: 50.5012
Epoch 4000, Loss: 50.5009
Epoch 5000, Loss: 50.5007
Epoch 6000, Loss: 50.5006
Epoch 7000, Loss: 50.5005
Epoch 8000, Loss: 50.5005
Epoch 9000, Loss: 50.5004
[[-163.43067114]]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

# ====== Step 1: Load arbitrary training data ======
# Example: Load from a CSV file
# CSV format should have three columns: x1, x2, y
try:
    df = pd.read_csv("training_data.csv")  # replace with your file path
except FileNotFoundError:
    # Fallback: manually define small dataset
    print("training_data.csv not found. Using fallback example data.")
    data = np.array([
        [0.4, -2, -10],
        [0.2, -1, -5],
        [0, 0, 0.0],
        [-0.2, 1, 5.0]
    ])
    df = pd.DataFrame(data, columns=["x1", "x2", "y"])

x_data = df[["x1", "x2"]].values.astype(np.float32)
y_data = df[["y"]].values.astype(np.float32)

x_tensor = torch.tensor(x_data)
y_tensor = torch.tensor(y_data)

# ====== Step 2: Define MLP ======
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.net(x)

model = MLP()

# ====== Step 3: Train the model ======
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(300):
    model.train()
    optimizer.zero_grad()
    predictions = model(x_tensor)
    loss = criterion(predictions, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# ====== Step 4: Predict on custom inputs ======
def predict_custom(x1, x2):
    model.eval()
    input_tensor = torch.tensor([[x1, x2]], dtype=torch.float32)
    with torch.no_grad():
        result = model(input_tensor)
    return result.item()

# ====== Step 5: CLI for testing ======
print("\nModel trained. Type two numbers (x1, x2) to predict y.")
print("Type 'q' to quit.")

while True:
    user_input = input("Enter x1, x2: ").strip()
    if user_input.lower() == "q":
        break
    try:
        x1, x2 = map(float, user_input.split(","))
        y_pred = predict_custom(x1, x2)
        print(f"Predicted y: {y_pred:.4f}")
    except:
        print("Invalid input. Format: x1, x2 (e.g., 1.5, -2.3)")


training_data.csv not found. Using fallback example data.
Epoch 50, Loss: 0.4895
Epoch 100, Loss: 0.0746
Epoch 150, Loss: 0.0067
Epoch 200, Loss: 0.0001
Epoch 250, Loss: 0.0000
Epoch 300, Loss: 0.0000

Model trained. Type two numbers (x1, x2) to predict y.
Type 'q' to quit.


Enter x1, x2:  0.2, -1


Predicted y: -4.9999
